# 1. ENTRENAMIENTO

In [135]:
#Importando librerias
import random
import json
import pickle
import numpy as np 

#Importar librerias necesarias para lematizar
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')

#Importar librerias necesarias para la red neuronal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [136]:
#Definiendo la funcion de  lematizacion de raiz
lemmatizer = WordNetLemmatizer()

In [137]:
#Lectura del archivo json para la bolsa de palabras
intents = json.loads(open('intents.json').read())

In [138]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents ['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if  intent['tag'] not in classes:
            classes.append(intent['tag'])


In [139]:
#Eliminar duplicados y lematizar
nltk.download('wordnet')
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pk1','wb'))
pickle.dump(classes, open('classes.pk1','wb'))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [140]:
#Creamos una bolsa de palabras
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [141]:
#Entrenando el modelo
random.shuffle(training)
training = np. array(training)

#Definiendo valores de enternamiento
train_x = list(training[:,0])
train_y = list(training[:,1])


In [142]:
#Creando una red neural

model = Sequential()
model.add(Dense(64,input_shape=(len(train_x[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation = 'softmax'))

sgd = SGD(lr=0.01, decay= 1e-6,momentum = 0.9, nesterov = True)

#Defininendo la compilacion y la parte que se entrenará
model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y),epochs=256, batch_size =5)
model.save('chatbot_model.model')
print('hecho')

Epoch 1/256
18/18 [==============================] - 0s 2ms/step - loss: 2.8097 - accuracy: 0.0581
Epoch 2/256
18/18 [==============================] - 0s 1ms/step - loss: 2.7543 - accuracy: 0.0698
Epoch 3/256
18/18 [==============================] - 0s 1ms/step - loss: 2.7390 - accuracy: 0.0930
Epoch 4/256
18/18 [==============================] - 0s 1ms/step - loss: 2.6766 - accuracy: 0.1860
Epoch 5/256
18/18 [==============================] - 0s 2ms/step - loss: 2.6759 - accuracy: 0.1512
Epoch 6/256
18/18 [==============================] - 0s 1ms/step - loss: 2.6326 - accuracy: 0.1977
Epoch 7/256
18/18 [==============================] - 0s 2ms/step - loss: 2.6642 - accuracy: 0.1860
Epoch 8/256
18/18 [==============================] - 0s 1ms/step - loss: 2.6085 - accuracy: 0.2209
Epoch 9/256
18/18 [==============================] - 0s 1ms/step - loss: 2.5689 - accuracy: 0.1628
Epoch 10/256
18/18 [==============================] - 0s 1ms/step - loss: 2.6167 - accuracy: 0.1628
Epoch 11/

# 2. MODELO CHATBOT

In [143]:
#Importar librerias para el modelo de chatbot
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [144]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

#Cargando archivos de forma binaria
words = pickle.load(open('words.pk1','rb'))
classes = pickle.load(open('classes.pk1','rb'))

#Cargamos el modelo
model = load_model('chatbot_model.model')
 

In [145]:
#Funcion para limpiar oraciones
def clean_up_sentences(sentece):
    sentece_words = nltk.word_tokenize(sentece)
    sentece_words = [lemmatizer.lemmatize(word) for word in sentece_words]
    return sentece_words

def bag_of_words(sentece):
    sentece_words = clean_up_sentences(sentece)
    bag = [0]* len(words)
    for w in sentece_words:
        for i, word in enumerate (words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [146]:
#Funcion para predecir respuesta
def predict_class(sentece):
    bow = bag_of_words(sentece)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    result = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD ]

    result.sort(key =lambda x: x[1],reverse= True)
    return_list=[]
    for r in result:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

In [148]:
#Funcion de respuesta e interfaz del bot
import pandas as pd
def get_response(intent_list,intent_json):
    tag = intent_list[0]['intent']
    list_of_intents = intent_json['intents']
    for i in  list_of_intents:
        if i['tag'] == tag:
            result =random.choice(i['responses'])
            break
    return result

print("Hola soy BOTITO")
acum = []
acum2 = []
contar = 0
while True:

    message = input("")
    if message == 'salir':
      break
    ints = predict_class(message)
    res = get_response(ints,intents)
    print(res)
    contar = contar + 1
 
    acum = acum +[message]
    
#Se crea un registro de todo lo hablado
data = {'mensaje': acum}
df1 = pd.DataFrame(data, columns = ['mensaje'])
df1.to_csv('example.csv')



Hola soy BOTITO
hola
Hola!
estoy triste
Estar triste disminuye las energias en el cuerpo
aea
Hola :)
oinoin
Hi!
yo no


IndexError: ignored

# 3. ANALIZADOR DE SENTIMIENTO

In [151]:
#Instalar libreria spanish_sentiment_analysis
pip install spanish_sentiment_analysis

SyntaxError: ignored

In [150]:
from classifier import SentimentClassifier
import statistics as stats

In [152]:
clf = SentimentClassifier()
acumulador = []
print("Hola soy BOTITO y pienso que este es un: ")
for x in range(contar):
  sentimiento = clf.predict(acum[x])
  if sentimiento > 0.65:
    print(">> Mensaje bueno: [" + acum[x] + "]")
    acumulador = acumulador + [sentimiento]

  elif sentimiento < 0.35:
    print(">> Mensaje malo: [" + acum[x] + "]")
    acumulador = acumulador + [sentimiento]
  else:
    print(">> Mensaje neutral: [" + acum[x] + "]")
    acumulador = acumulador + [sentimiento]
#for y in range(contar):
sentimientoFinal = stats.mean(acumulador)
print('Nivel de sentimiento total: ')
if sentimientoFinal > 0.65:
    print(">>Sentimiento Bueno")
    
elif sentimientoFinal < 0.35:
    print(">>Sentimiento Malo")
    
else:
    print(">>Sentimiento neutral")
    


Hola soy BOTITO y pienso que este es un: 
>> Mensaje neutral: [hola]
>> Mensaje malo: [estoy triste]
>> Mensaje neutral: [aea]
>> Mensaje neutral: [oinoin]
Nivel de sentimiento total: 
>>Sentimiento neutral
